In [1]:
import json
import pickle
import random

import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.nn.utils.rnn as rnn_utils

import nltk
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
import time

import copy

from Vocab import Vocab

import torch
torch.cuda.set_device(1)

print('import over')

import over


In [2]:
def batch_words2sentence(words_list):
    return [' '.join(words) for words in words_list]
def batch_tokens2words(tokens_list, vocab):
    ##    para: tokens_list is list[list] type
    ##    return: words_list corresponding to tokens
    return [[vocab.token2word[token] for token in tokens] for tokens in tokens_list]

def batch_tokens_remove_eos(tokens_list, vocab):
    ##    para: tokens_list is list[list] type
    ##    return pure tokens_list removed eos symbol
    result=[]
    for tokens in tokens_list:
        tokens_filtered=[]
        for token in tokens:
            if token == vocab.word2token['<eos>']:
#                 tokens_filtered.append(token)
                break
            else:
                tokens_filtered.append(token)
        result.append(tokens_filtered)
    return result

def batch_tokens_bleu(references, candidates, smooth_epsilon=0.001):
    ##    para: references and candidates are list[list] type
    ##    return: list of BLEU for every sample
    ##
    bleu_scores=[]
    for ref, candidate in zip(references, candidates):
        if min(len(ref), len(candidate))<4:
            bleu_scores.append(0)
        else:
            bleu_scores.append(sentence_bleu([ref], candidate, smoothing_function = SmoothingFunction(epsilon=smooth_epsilon).method1))
    return bleu_scores

with open('data_set/vocab.pk', 'rb') as f:
    vocab=pickle.load(f)

    
def seqs_split(seqs, vocab):
    seqs = batch_tokens_remove_eos(seqs, vocab)
    simple_sent1s=[]
    simple_sent2s=[]
    for seq in seqs:
        simple_sent1=[]
        simple_sent2=[]
        sent=simple_sent1
        for token in seq:
            if token==vocab.word2token['<split>']:
                sent=simple_sent2
            else:
                sent.append(token)
        simple_sent1s.append(simple_sent1)
        simple_sent2s.append(simple_sent2)
        
    return simple_sent1s, simple_sent2s

def simple_sents_concat(simple_sent1s, simple_sent2s, vocab, max_length):
    simple_sent_lens=[]
    simple_sents=simple_sent1s
    for i, sent in enumerate(simple_sent2s):
        simple_sents[i].append(vocab.word2token['<split>'])
        for token in sent:
            simple_sents[i].append(token)

        #if there is no <split> in simple_sent1s and simple_sent2s, then the length of sents_concat will be longer than max_length
        if len(simple_sents[i])>max_length:
            simple_sents[i] = simple_sents[i][:max_length]
            
        simple_sent_lens.append(len(simple_sents[i]))
            
        while(len(simple_sents[i])<max_length):
            simple_sents[i].append(vocab.word2token['<padding>'])
            
    return simple_sents, simple_sent_lens


def get_lm_inputs_and_labels(sents, vocab, max_length):
    lm_inputs=copy.deepcopy(sents)
    lm_labels=copy.deepcopy(sents)
    lm_input_lens=[]
    
    for sent in lm_inputs:
        if len(sent)>=max_length:
            sent=sent[:max_length-1]
        sent.insert(0, vocab.word2token['<sos>'])
        lm_input_lens.append(len(sent))
        while(len(sent)<max_length):
            sent.append(vocab.word2token['<padding>'])

    for sent in lm_labels:
        if len(sent)>=max_length:
            sent = sent[:max_length-1]
        sent.append(vocab.word2token['<eos>'])
        while(len(sent)<max_length):
            sent.append(vocab.word2token['<padding>'])
        
    return lm_inputs, lm_input_lens, lm_labels


def duplicate_reconstruct_labels(sents, topk):
    return [x for x in sents for ii in range(topk)]


def batch_tokens_bleu_split_version(references, candidates, vocab, smooth_epsilon=0.001):
    # needn't remove '<sos>' token before calling this function, which is different from the 'batch_token_bleu()' version
    #
    ref1, ref2 = seqs_split(references, vocab)
    cand1, cand2 = seqs_split(candidates, vocab)
    bleu_simple_sent1s = batch_tokens_bleu(ref1, cand1)
    bleu_simple_sent2s = batch_tokens_bleu(ref2, cand2)
#     print(bleu_simple_sent1s)
#     print(bleu_simple_sent2s)
    bleu=[]
    for idx in range(len(bleu_simple_sent1s)):
        bleu.append((bleu_simple_sent1s[idx]+bleu_simple_sent2s[idx])/2)
    return bleu


def set_model_grad(model, is_grad):
    for param in model.parameters():
         param.requires_grad = is_grad

In [3]:
with open('./data_set/split_data_set/validation_labels.pk', 'rb') as f:
    split_valid_set_labels = pickle.load(f)
with open('./data_set/split_data_set/validation_predicts-topk=3.pk', 'rb') as f:
    split_valid_set_predicts = pickle.load(f)
    
    
print(len(split_valid_set_labels), len(split_valid_set_predicts))

5000 15000


In [4]:
class LanguageModel(nn.Module):
    def __init__(self, use_cuda, hidden_dim, input_dim, vocab):#, pre_train_weight, is_fix_word_vector = 1):
        super(LanguageModel, self).__init__()
        
        self.use_cuda = use_cuda
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim
        self.vocab = vocab
        
        self.lstm=torch.nn.LSTM(input_size=self.input_dim, 
                                hidden_size= self.hidden_dim, 
                                bidirectional=False,
                                batch_first=True
                               )
        
        #embedding
        self.embed=nn.Embedding(len(self.vocab.word2token), input_dim)
        #loading pre trained word embedding
        with open('data_set/pre_trained_token_embedding_300d.pk', 'rb') as f:
            pre_train_word_embedding = pickle.load(f)
            
        self.embed.weight.data.copy_(torch.FloatTensor(pre_train_word_embedding))
#         self.embed.weight.requires_grad = False


        self.weight = [1]*len(self.vocab.word2token)
        self.weight[self.vocab.word2token['<padding>']]=0
        self.cost_func = nn.CrossEntropyLoss(weight=torch.Tensor(self.weight), reduce=True)
        self.fcnn=nn.Linear(in_features = self.hidden_dim, out_features = len(self.vocab.word2token))
        
        self.log_softmax = nn.LogSoftmax(dim=1)
        
    def order(self, inputs, inputs_len):    #inputs: tensor, inputs_len: 1D tensor
        inputs_len, sort_ids = torch.sort(inputs_len, dim=0, descending=True)
        
        if self.use_cuda:
            inputs = inputs.index_select(0, Variable(sort_ids).cuda())
        else:
            inputs = inputs.index_select(0, Variable(sort_ids))
        
        _, true_order_ids = torch.sort(sort_ids, dim=0, descending=False)
        
        return inputs, inputs_len, true_order_ids
    #
    def forward(self, inputs, inputs_len):
        inputs = Variable(inputs)
        if self.use_cuda:
            inputs=inputs.cuda()
            
        inputs, sort_len, true_order_ids = self.order(inputs, inputs_len)

        in_vecs=self.embed(inputs)

        packed = rnn_utils.pack_padded_sequence(input=in_vecs, lengths=list(sort_len), batch_first =True)
        
        outputs, (hn,cn) = self.lstm(packed)
        outputs, sent_lens = rnn_utils.pad_packed_sequence(outputs)
        
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        outputs = outputs.transpose(0,1)  #transpose is necessary
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        
        #warnning: outputs, hn and cn have been sorted by sentences length so the order is wrong, now to sort them
        if self.use_cuda:
            outputs = outputs.index_select(0, Variable(true_order_ids).cuda())
        else:
            outputs = outputs.index_select(0, Variable(true_order_ids))
        
#         hn = torch.cat((hn[0], hn[1]), dim=1)
#         cn = torch.cat((cn[0], cn[1]), dim=1)
#         #print('hn size and cn size: ', hn.size(), cn.size())
        
#         if self.use_cuda:
#             hn = hn.index_select(0, Variable(true_order_ids).cuda())
#             cn = cn.index_select(0, Variable(true_order_ids).cuda())
#         else:
#             hn = hn.index_select(0, Variable(true_order_ids))
#             cn = cn.index_select(0, Variable(true_order_ids))
        logits = self.fcnn(outputs)
        return logits
    
    def get_loss(self, logits, labels):
        labels = self._tocuda(Variable(labels))
        sent_len = logits.size(dim=1)
        labels = labels[:, :sent_len]
        labels = labels.contiguous().view(-1)
        logits = logits.view(-1, len(self.vocab.word2token))
#         print('logits size: ', logits.size())
        
        return self.cost_func(logits, labels)
    
    def get_sentences_ppl(self, inputs, inputs_len, labels):
        
        vocab_size=len(self.vocab.word2token)
        batch_size=inputs.size(0)
        
        logits = self.forward(inputs, inputs_len)
        
        labels = self._tocuda(Variable(labels))
        sent_len = logits.size(dim=1)
        labels = labels[:, :sent_len]
        labels = labels.contiguous()
        logits = logits.view(-1, vocab_size)
        log_probs= self.log_softmax(logits).view(-1)
        
        pos_bias=torch.LongTensor([i*vocab_size for i in range(sent_len)]).view(1,-1)
        pos_bias = pos_bias.expand(batch_size, pos_bias.size(1))
#         print(pos_bias.size())

        batch_bias = torch.LongTensor([i*vocab_size*sent_len for i in range(batch_size)]).view(-1,1)
        batch_bias = batch_bias.expand(batch_bias.size(0), sent_len)
#         print(batch_bias.size())

        pos_bias = self._tocuda(Variable(pos_bias, requires_grad=0))
        batch_bias = self._tocuda(Variable(batch_bias, requires_grad=0))
        
        indices=labels+pos_bias+batch_bias
        indices = indices.view(-1)

        results=log_probs[indices].view(batch_size, sent_len)
        
        sents_ppl=[]
        for idx, result in enumerate(results):
            mean_log_prob = torch.mean(result[:inputs_len[idx]])
            sents_ppl.append(mean_log_prob)

        sents_ppl = torch.cat(sents_ppl, dim=0)
        sents_ppl = torch.exp(-sents_ppl)
        
        if self.use_cuda:
            return sents_ppl.cpu().data.tolist()
        else:
            return sents_ppl.data.tolist()
        
    def _tocuda(self, var):
        if self.use_cuda:
            return var.cuda()
        else:
            return var

In [5]:
use_cuda = 1
hidden_dim = 512
input_dim = 300
lr=0.005
batch_size=100

epochs=10000
train_bleu_mean=-1
train_bleu_max=-1

language_model = LanguageModel(use_cuda = use_cuda, input_dim = input_dim, hidden_dim = hidden_dim, vocab = vocab)
#512
model_path = './models_language_model/time-[2019-02-26-10-13-36]-info=[language_model]-loss=4.188151360-bleu=-1.0000-hidden_dim=512-input_dim=300-epoch=19-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
model_path = './models_language_model/time-[2019-02-26-11-27-36]-info=[language_model]-loss=3.414012194-bleu=-1.0000-hidden_dim=512-input_dim=300-epoch=21-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
model_path = './models_language_model/time-[2019-02-26-13-18-56]-info=[language_model]-loss=4.003012180-bleu=-1.0000-hidden_dim=512-input_dim=300-epoch=24-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
#2048
# model_path = './models_language_model/time-[2019-02-28-02-41-29]-info=[language_model]-loss=4.125045300-bleu=-1.0000-hidden_dim=2048-input_dim=300-epoch=2-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
# model_path = './models_language_model/time-[2019-02-28-07-04-08]-info=[language_model]-loss=3.475848675-bleu=-1.0000-hidden_dim=2048-input_dim=300-epoch=4-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
# model_path = './models_language_model/time-[2019-02-28-09-15-24]-info=[language_model]-loss=4.169310093-bleu=-1.0000-hidden_dim=2048-input_dim=300-epoch=5-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
# model_path = './models_language_model/time-[2019-02-28-13-37-55]-info=[language_model]-loss=4.279534817-bleu=-1.0000-hidden_dim=2048-input_dim=300-epoch=7-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
#1024
# model_path = './models_language_model/time-[2019-02-27-16-28-32]-info=[language_model]-loss=4.221434593-bleu=-1.0000-hidden_dim=1024-input_dim=300-epoch=1-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
# mdoel_path = './models_language_model/time-[2019-02-27-18-39-43]-info=[language_model]-loss=4.328499317-bleu=-1.0000-hidden_dim=1024-input_dim=300-epoch=3-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'
# model_path = './models_language_model/time-[2019-02-27-21-58-23]-info=[language_model]-loss=4.111208439-bleu=-1.0000-hidden_dim=1024-input_dim=300-epoch=6-batch_size=100-batch_id=[1-[of]-9899]-lr=0.0050'

pre_train = torch.load(model_path, map_location='cpu')
language_model.load_state_dict(pre_train)

if use_cuda:
    language_model = language_model.cuda()

In [8]:
batch_id=0
all_rewards=[]
for idx in range(0, len(split_valid_set_labels), batch_size):
    batch_id+=1
    simple_sent1s, simple_sent2s = seqs_split(split_valid_set_labels[idx:idx+batch_size], vocab)
        
    lm_input1s, lm_input1_lens, lm_label1s = get_lm_inputs_and_labels(simple_sent1s, vocab, 61)
    simple_sent1s_ppl = language_model.get_sentences_ppl(torch.LongTensor(lm_input1s), 
                                                  torch.LongTensor(lm_input1_lens), 
                                                  torch.LongTensor(lm_label1s)
                                                )
    lm_input2s, lm_input2_lens, lm_label2s = get_lm_inputs_and_labels(simple_sent2s, vocab, 61)
    simple_sent2s_ppl = language_model.get_sentences_ppl(torch.LongTensor(lm_input2s), 
                                                  torch.LongTensor(lm_input2_lens), 
                                                  torch.LongTensor(lm_label2s)
                                                )
    for ii in range(len(simple_sent1s_ppl)):
        all_rewards.append((1/simple_sent1s_ppl[ii]+1/simple_sent2s_ppl[ii])/2)
        
print(batch_id)
s=0
for reward in all_rewards:
    s+=reward
print(1/(s/len(all_rewards)))

50
67.02538807309371


In [7]:
batch_id=0
all_rewards=[]

predicts=[]
for ii in range(len(split_valid_set_predicts)):
    if ii%3==0:
        predicts.append(split_valid_set_predicts[ii])
split_valid_set_predicts=predicts
for idx in range(0, len(split_valid_set_predicts), batch_size):
    batch_id+=1
    simple_sent1s, simple_sent2s = seqs_split(split_valid_set_predicts[idx:idx+batch_size], vocab)
        
    lm_input1s, lm_input1_lens, lm_label1s = get_lm_inputs_and_labels(simple_sent1s, vocab, 61)
    simple_sent1s_ppl = language_model.get_sentences_ppl(torch.LongTensor(lm_input1s), 
                                                  torch.LongTensor(lm_input1_lens), 
                                                  torch.LongTensor(lm_label1s)
                                                )
    lm_input2s, lm_input2_lens, lm_label2s = get_lm_inputs_and_labels(simple_sent2s, vocab, 61)
    simple_sent2s_ppl = language_model.get_sentences_ppl(torch.LongTensor(lm_input2s), 
                                                  torch.LongTensor(lm_input2_lens), 
                                                  torch.LongTensor(lm_label2s)
                                                )
    for ii in range(len(simple_sent1s_ppl)):
        all_rewards.append((1/simple_sent1s_ppl[ii]+1/simple_sent2s_ppl[ii])/2)
        
print(batch_id)
s=0
for reward in all_rewards:
    s+=reward
print(1/(s/len(all_rewards)))

50
101.69841964367605


In [95]:
data_range = int((len(split_valid_set_inputs)-1))
rand_idx=random.randint(0, data_range)
sample_num=2
language_model.eval()
sents_ppl = language_model.get_sentences_ppl(torch.LongTensor(split_valid_set_inputs[rand_idx:rand_idx+sample_num]), 
                                          torch.LongTensor(split_valid_set_input_lens[rand_idx:rand_idx+sample_num]),
                                          torch.LongTensor(split_valid_set_labels[rand_idx:rand_idx+sample_num])
                                          )

print(sents_ppl)


logits = language_model.forward(torch.LongTensor(split_valid_set_inputs[rand_idx:rand_idx+sample_num]), 
                                     torch.LongTensor(split_valid_set_input_lens[rand_idx:rand_idx+sample_num]),
                                                )

_, predicts = logits.max(dim=2)
predicts = predicts.cpu().data.tolist()
predicts = batch_tokens_remove_eos(tokens_list=predicts, vocab=vocab)
words_list = batch_tokens2words(predicts, vocab)
sents = batch_words2sentence(words_list=words_list)
for sent in sents:
    print(sent)

print('\n')

#labels
labels=split_valid_set_labels[rand_idx:rand_idx+sample_num]
labels = batch_tokens_remove_eos(tokens_list=labels, vocab=vocab)
words_list = batch_tokens2words(labels, vocab)
sents = batch_words2sentence(words_list=words_list)
for sent in sents:
    print(sent)
    
language_model.train()
print('\n')

[452.750244140625, 56.67298889160156]
<low_freq> the <low_freq> 's , to <low_freq> slaves , the <low_freq> is <low_freq> were to and led been a a is now the - <low_freq> , , and been different - ,
<low_freq> the <low_freq> was the <low_freq> , of the , the were were not as the <low_freq> of of the , and the the province population of the <low_freq> of china . the , <low_freq> . and is the part minorities separate of the province


since the slave masters spoke southern american english , the english the slaves learned , which has developed into what is now african american vernacular english , had many sae features .
since the <low_freq> of the taiwan province in 1998 , these subdivisions are regarded as the principal subdivisions of taiwan , and cover the current jurisdiction of the republic of china except <low_freq> and <low_freq> , which are historically and administratively part of fujian .




In [96]:
def model_test(model, batch_size, dataset_size, test_set_inputs, test_set_input_lens, test_set_labels):
    ppl_sum=0
    all_ppl=[]
    for start_idx in range(0, dataset_size, batch_size):
        sents_ppl = language_model.get_sentences_ppl(torch.LongTensor(test_set_inputs[start_idx:start_idx+batch_size]), 
                                                      torch.LongTensor(test_set_input_lens[start_idx:start_idx+batch_size]), 
                                                      torch.LongTensor(test_set_labels[start_idx:start_idx+batch_size])
                                                    )
        for p in sents_ppl:
            ppl_sum+=p
            all_ppl.append(p)
        
    return ppl_sum/dataset_size, all_ppl

batch_size = 200
dataset_size=len(split_valid_set_inputs)
ppl_mean, all_ppl = model_test(language_model, batch_size, dataset_size, split_valid_set_inputs, split_valid_set_input_lens, split_valid_set_labels)
print(ppl_mean)

177.02750500671863


In [98]:
def mess_tokens_up(test_set_inputs, test_set_input_lens, test_set_labels, mess_times):
    mess_inputs=copy.deepcopy(test_set_inputs)
    mess_labels=copy.deepcopy(test_set_labels)
    mess_input_lens=copy.deepcopy(test_set_input_lens)
    
    for ii, (mess_input, mess_label) in enumerate(zip(mess_inputs, mess_labels)):
        for mess_time in range(mess_times):
            start_idx=random.randint(1, test_set_input_lens[ii]-1)
            end_idx=random.randint(1, test_set_input_lens[ii]-1)
            mess_input[start_idx], mess_input[end_idx]=mess_input[end_idx], mess_input[start_idx]
            mess_label[start_idx-1], mess_label[end_idx-1]=mess_label[end_idx-1], mess_label[start_idx-1]
    return mess_inputs, mess_input_lens, mess_labels

mess_inputs, mess_input_lens, mess_labels = mess_tokens_up(split_valid_set_inputs, split_valid_set_input_lens, 
                                                           split_valid_set_labels, mess_times=2)
print('mess sents up. ')

ppl_mean, all_ppl = model_test(language_model, batch_size, dataset_size, mess_inputs, mess_input_lens, mess_labels)
print(ppl_mean)

mess sents up. 
904.5610524456024


In [97]:
import scipy
from scipy.stats import pearsonr

x = scipy.array(all_ppl)
y = scipy.array(split_valid_set_input_lens)

r_row, p_value = pearsonr(x, y)
print(r_row, p_value)

-0.0347608346071 0.0139677994815
